#### Config Groups

A `Config Group` is a named group with a set of valid options. Selecting a non-existent config option generates an error message with the valid options.

Since we moved all the configs into the `conf` directory, we need to tell Hydra where to find them using the config_path parameter. `config_path` <b>is a directory relative to</b> `my_app.py.`

```
from omegaconf import DictConfig, OmegaConf
import hydra

@hydra.main(version_base=None, config_path="conf")
def my_app(cfg: DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))

if __name__ == "__main__":
    my_app()
```

Running `my_app.py` without requesting a configuration will print an empty config.

```
$ python my_app.py
{}
```

Select an item from a config group with `+GROUP=OPTION`, e.g:

```
$ python my_app.py +db=postgresql
db:
  driver: postgresql
  pass: drowssap
  timeout: 10
  user: postgres_user
```

By default, the config group determines where the config content is placed inside the final config object. In Hydra, the path to the config content is referred to as the config `package`. The package of `db/postgresql.yaml` is `db`:

Like before, you can still override individual values in the resulting config:

```
$ python my_app.py +db=postgresql db.timeout=20
db:
  driver: postgresql
  pass: drowssap
  timeout: 20
  user: postgres_user
```

#### Selecting Default Configs

You can have multiple items in the defaults list, e.g: 

```
defaults:
 - db: mysql
 - db/mysql/engine: innodb
```

The defaults are ordered:

- If multiple configs define the same value, the last one wins.
- If multiple configs contribute to the same dictionary, the result is the combined dictionary.

<b>Overriding a config group default</b>

_You can still load PostgreSQL, and override individual values._

```
$ python my_app.py db=postgresql db.timeout=20
db:
  driver: postgresql
  pass: drowssap
  timeout: 20
  user: postgres_user
```

_You can remove a default entry from the defaults list by prefixing it with ~:_

```
$ python my_app.py ~db
{}
```

<b>Composition Order of Primary Config</b>

Your primary config can contain both config values and a Defaults List. In such cases, you should add the `_self_` keyword to your defaults list to specify the composition order of the config file relative to the items in the defaults list.

- If you want your primary config to override the values of configs from the Defaults List, append `_self_` to the end of the Defaults List.
- If you want the configs from the Defaults List to override the values in your primary config, insert `_self_` as the first item in your Defaults List.

_config.yaml_

```
defaults:
  - db: mysql
  - _self_

db:
  user: root
```

_Result config:db.user from config.yaml_

```
db:
  driver: mysql  # db/mysql.yaml
  pass: secret   # db/mysql.yaml 
  user: root     # config.yaml
```

_config.yaml_



```
defaults:
  - _self_
  - db: mysql

db:
  user: root
```

_Result config: All values from db/mysql_

```
db:
  driver: mysql # db/mysql.yaml
  pass: secret  # db/mysql.yaml
  user: omry    # db/mysql.yaml
```